In [1]:
import os

In [2]:
%pwd

'c:\\Users\\vishw\\Documents\\college_projects\\Obesity-Predictor\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\vishw\\Documents\\college_projects\\Obesity-Predictor'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    AWS_REGION: str
    BUCKET_NAME: str
    S3_OBJECT_KEY: str
    root_dir: Path
    LOCAL_DOWNLOAD_FILE: Path  # ✅ Added file path


In [6]:
from ObesityPredictor.constants import *
from ObesityPredictor.utils.common import read_yaml, create_directories

In [ ]:
from pathlib import Path
#from ObesityPredictor.entity.config_entity import DataIngestionConfig  # Ensure correct import

class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Ensure the root directory exists
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # Ensure the data ingestion directory exists
        create_directories([config.root_dir])

        # ✅ Correctly instantiate DataIngestionConfig with `LOCAL_DOWNLOAD_FILE`
        data_ingestion_config = DataIngestionConfig(
            AWS_REGION=config.AWS_REGION,
            BUCKET_NAME=config.BUCKET_NAME,
            S3_OBJECT_KEY=config.S3_OBJECT_KEY,
            root_dir=Path(config.root_dir),
            LOCAL_DOWNLOAD_FILE=Path(config.LOCAL_DOWNLOAD_FILE)  
        )

        return data_ingestion_config



In [8]:
import boto3
import os
from pathlib import Path
from ObesityPredictor import logger

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.s3_client = boto3.client("s3", region_name=self.config.AWS_REGION)

    def download_file_from_s3(self) -> str:
        """
        Download a file from AWS S3 to the local system.

        Returns:
            str: Path to the downloaded file.
        """
        try:
            bucket_name = self.config.BUCKET_NAME
            s3_object_key = self.config.S3_OBJECT_KEY
            local_download_path = Path(self.config.LOCAL_DOWNLOAD_FILE)

            # Ensure the directory exists
            os.makedirs(local_download_path.parent, exist_ok=True)

            # ✅ Fix: Use `LOCAL_DOWNLOAD_FILE` instead of `root_dir`
            logger.info(f"Downloading {s3_object_key} from S3 bucket {bucket_name} to {local_download_path}")

            # Download file from S3
            self.s3_client.download_file(bucket_name, s3_object_key, str(local_download_path))

            logger.info(f"File downloaded successfully: {local_download_path}")

            return str(local_download_path)

        except Exception as e:
            logger.error(f"Error downloading file from S3: {str(e)}")
            raise e


In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file_from_s3()
except Exception as e:
    raise e

[2025-02-13 16:32:28,009: INFO: common: yaml file: config\config.yaml loaded successfully]


ValueError: yaml file is empty

In [10]:
%pwd

'c:\\Users\\vishw\\Documents\\college_projects\\Obesity-Predictor'